# Solution: CRUD Operations

## ⚠️ Try the exercise first!

**Don't look at this solution until you've attempted the exercise yourself!**

## Example Solutions

Here are example solutions for the CRUD operations exercises:


In [ ]:
# Exercise 1: Setup and Model Creation Solution

# Setup
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine
engine = create_engine('sqlite:///crud_exercise.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

print("SQLAlchemy setup complete!")


In [ ]:
# Define models
class Company(Base):
    __tablename__ = 'companies'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    industry = Column(String(50))
    founded_year = Column(Integer)
    is_public = Column(Boolean, default=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # One-to-many relationship with employees
    employees = relationship("Employee", back_populates="company", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Company(name='{self.name}', industry='{self.industry}')>"

class Employee(Base):
    __tablename__ = 'employees'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    position = Column(String(50))
    salary = Column(Float)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign key to companies table
    company_id = Column(Integer, ForeignKey('companies.id'), nullable=False)
    
    # Many-to-one relationship with company
    company = relationship("Company", back_populates="employees")
    
    def __repr__(self):
        return f"<Employee(name='{self.name}', position='{self.position}')>"

# Create tables
Base.metadata.create_all(engine)

print("Models defined and tables created!")


In [ ]:
# Exercise 2: CREATE Operations Solution

# Create a session
session = Session()

# Create companies using session.add()
company1 = Company(
    name='TechCorp',
    industry='Technology',
    founded_year=2010,
    is_public=True
)
session.add(company1)

company2 = Company(
    name='FinanceInc',
    industry='Finance',
    founded_year=2005,
    is_public=False
)
session.add(company2)

company3 = Company(
    name='HealthCare Plus',
    industry='Healthcare',
    founded_year=2015,
    is_public=True
)
session.add(company3)

session.commit()
print("Created 3 companies using session.add()")

# Create employees using session.add_all()
employees_data = [
    Employee(name='Alice Johnson', position='Software Engineer', salary=75000, company_id=company1.id),
    Employee(name='Bob Smith', position='Data Scientist', salary=85000, company_id=company1.id),
    Employee(name='Charlie Brown', position='Financial Analyst', salary=65000, company_id=company2.id),
    Employee(name='Diana Prince', position='Investment Manager', salary=95000, company_id=company2.id),
    Employee(name='Eve Wilson', position='Nurse', salary=55000, company_id=company3.id)
]

session.add_all(employees_data)
session.commit()
print("Created 5 employees using session.add_all()")

# Use bulk_save_objects() for more employees
more_employees = [
    Employee(name='Frank Miller', position='DevOps Engineer', salary=80000, company_id=company1.id),
    Employee(name='Grace Lee', position='Product Manager', salary=90000, company_id=company1.id),
    Employee(name='Henry Davis', position='Doctor', salary=120000, company_id=company3.id)
]

session.bulk_save_objects(more_employees)
session.commit()
print("Created 3 more employees using bulk_save_objects()")


In [ ]:
# Exercise 3: READ Operations Solution

# Basic queries
print("=== Basic Queries ===")
all_companies = session.query(Company).all()
print(f"All companies: {len(all_companies)}")
for company in all_companies:
    print(f"  {company}")

all_employees = session.query(Employee).all()
print(f"\nAll employees: {len(all_employees)}")
for employee in all_employees:
    print(f"  {employee}")

# Filtering
print("\n=== Filtering ===")
tech_companies = session.query(Company).filter(Company.industry == 'Technology').all()
print("Technology companies:")
for company in tech_companies:
    print(f"  {company.name}")

high_salary_employees = session.query(Employee).filter(Employee.salary > 80000).all()
print("\nHigh salary employees (>$80k):")
for employee in high_salary_employees:
    print(f"  {employee.name} - ${employee.salary}")

# Sorting
print("\n=== Sorting ===")
employees_by_salary = session.query(Employee).order_by(Employee.salary.desc()).all()
print("Employees sorted by salary (desc):")
for employee in employees_by_salary:
    print(f"  {employee.name} - ${employee.salary}")

# Pagination
print("\n=== Pagination ===")
first_two_employees = session.query(Employee).order_by(Employee.id).limit(2).all()
print("First 2 employees:")
for employee in first_two_employees:
    print(f"  {employee.name}")

# Relationships
print("\n=== Relationships ===")
techcorp = session.query(Company).filter(Company.name == 'TechCorp').first()
if techcorp:
    print(f"TechCorp employees: {len(techcorp.employees)}")
    for employee in techcorp.employees:
        print(f"  {employee.name} - {employee.position}")

# Joins
print("\n=== Joins ===")
employees_with_companies = session.query(Employee, Company).join(Company).filter(Employee.salary > 70000).all()
print("High-earning employees with company info:")
for employee, company in employees_with_companies:
    print(f"  {employee.name} at {company.name} - ${employee.salary}")


In [ ]:
# Advanced READ Operations Solution

from sqlalchemy import func

# Aggregation functions
print("=== Aggregation Functions ===")
total_companies = session.query(func.count(Company.id)).scalar()
total_employees = session.query(func.count(Employee.id)).scalar()
avg_salary = session.query(func.avg(Employee.salary)).scalar()
min_salary = session.query(func.min(Employee.salary)).scalar()
max_salary = session.query(func.max(Employee.salary)).scalar()

print(f"Total companies: {total_companies}")
print(f"Total employees: {total_employees}")
print(f"Average salary: ${avg_salary:.2f}")
print(f"Min salary: ${min_salary}")
print(f"Max salary: ${max_salary}")

# Group by operations
print("\n=== Group By Operations ===")
employees_per_company = session.query(
    Company.name,
    func.count(Employee.id).label('employee_count'),
    func.avg(Employee.salary).label('avg_salary')
).join(Employee).group_by(Company.id, Company.name).all()

print("Employees per company:")
for company_name, emp_count, avg_sal in employees_per_company:
    print(f"  {company_name}: {emp_count} employees, avg salary ${avg_sal:.2f}")

# Subqueries
print("\n=== Subqueries ===")
avg_salary_subquery = session.query(func.avg(Employee.salary)).scalar()
employees_above_avg = session.query(Employee).filter(
    Employee.salary > avg_salary_subquery
).all()

print(f"Employees above average salary (${avg_salary_subquery:.2f}):")
for employee in employees_above_avg:
    print(f"  {employee.name} - ${employee.salary}")

# Exists subquery
print("\n=== Exists Subquery ===")
companies_with_employees = session.query(Company).filter(
    session.query(Employee).filter(Employee.company_id == Company.id).exists()
).all()

print("Companies that have employees:")
for company in companies_with_employees:
    print(f"  {company.name}")


In [ ]:
# Exercise 4: UPDATE Operations Solution

# Update a company's attributes
print("=== Direct Attribute Update ===")
techcorp = session.query(Company).filter(Company.name == 'TechCorp').first()
if techcorp:
    print(f"Before update: {techcorp.name} - Founded: {techcorp.founded_year}")
    techcorp.founded_year = 2011
    techcorp.is_public = False
    session.commit()
    print(f"After update: {techcorp.name} - Founded: {techcorp.founded_year}, Public: {techcorp.is_public}")

# Bulk update
print("\n=== Bulk Update ===")
updated_count = session.query(Employee).filter(Employee.salary < 60000).update({
    Employee.salary: Employee.salary * 1.1  # 10% raise
}, synchronize_session=False)
session.commit()
print(f"Updated {updated_count} low-salary employees with 10% raise")

# Conditional update
print("\n=== Conditional Update ===")
tech_employees_updated = session.query(Employee).join(Company).filter(
    Company.industry == 'Technology'
).update({
    Employee.salary: Employee.salary * 1.05  # 5% raise for tech employees
}, synchronize_session=False)
session.commit()
print(f"Updated {tech_employees_updated} technology employees with 5% raise")


In [ ]:
# DELETE Operations Solution

# Delete a specific employee
print("=== Delete Specific Employee ===")
employee_to_delete = session.query(Employee).filter(Employee.name == 'Eve Wilson').first()
if employee_to_delete:
    print(f"Deleting employee: {employee_to_delete.name}")
    session.delete(employee_to_delete)
    session.commit()
    print("Employee deleted successfully")

# Bulk delete
print("\n=== Bulk Delete ===")
deleted_count = session.query(Employee).filter(Employee.salary < 60000).delete()
session.commit()
print(f"Deleted {deleted_count} low-salary employees")

# Test cascade delete
print("\n=== Cascade Delete Test ===")
# Create a test company with employees
test_company = Company(
    name='Test Company',
    industry='Testing',
    founded_year=2023
)
session.add(test_company)
session.commit()

test_employee = Employee(
    name='Test Employee',
    position='Tester',
    salary=50000,
    company_id=test_company.id
)
session.add(test_employee)
session.commit()

print(f"Created test company with employee: {test_company.name}")

# Delete the company (this will cascade delete the employee)
session.delete(test_company)
session.commit()

print("Test company and associated employee deleted (cascade)")

# Verify the employee was also deleted
remaining_test_employees = session.query(Employee).filter(Employee.name == 'Test Employee').count()
print(f"Remaining test employees: {remaining_test_employees}")


In [ ]:
# Transaction Management Solution

# Successful transaction
print("=== Successful Transaction ===")
try:
    with session.begin():
        new_company = Company(
            name='Transaction Test Company',
            industry='Testing',
            founded_year=2023
        )
        session.add(new_company)
        session.flush()  # Get the ID
        
        new_employee = Employee(
            name='Transaction Test Employee',
            position='Tester',
            salary=60000,
            company_id=new_company.id
        )
        session.add(new_employee)
        # This will succeed
        print("Transaction completed successfully")
except Exception as e:
    print(f"Error in successful transaction: {e}")

# Failed transaction with rollback
print("\n=== Failed Transaction with Rollback ===")
try:
    with session.begin():
        # Create a company
        failing_company = Company(
            name='Failing Company',
            industry='Testing',
            founded_year=2023
        )
        session.add(failing_company)
        session.flush()
        
        # Create an employee
        failing_employee = Employee(
            name='Failing Employee',
            position='Tester',
            salary=60000,
            company_id=failing_company.id
        )
        session.add(failing_employee)
        
        # Intentionally cause an error (duplicate company name)
        duplicate_company = Company(
            name='Failing Company',  # Same name - will cause error
            industry='Testing',
            founded_year=2023
        )
        session.add(duplicate_company)
        # This will fail and rollback automatically
        session.commit()
        
except Exception as e:
    print(f"Transaction failed as expected: {e}")
    print("Rollback handled automatically by context manager")

# Verify rollback worked
failing_company_exists = session.query(Company).filter(Company.name == 'Failing Company').count()
failing_employee_exists = session.query(Employee).filter(Employee.name == 'Failing Employee').count()
print(f"Failing company exists: {failing_company_exists}")
print(f"Failing employee exists: {failing_employee_exists}")

# Close the session
session.close()

print("\n" + "="*50)
print("CRUD Operations solution completed successfully!")
print("="*50)


## Key Learning Points

### CRUD Operations Mastery
- **CREATE**: Use `session.add()`, `session.add_all()`, and `bulk_save_objects()` appropriately
- **READ**: Master filtering, sorting, pagination, and relationship queries
- **UPDATE**: Use direct updates, bulk updates, and conditional updates
- **DELETE**: Handle single deletes, bulk deletes, and cascade deletes

### Advanced Query Techniques
- **Aggregations**: Use `func.count()`, `func.avg()`, `func.min()`, `func.max()`
- **Grouping**: Use `group_by()` for summary statistics
- **Subqueries**: Use scalar subqueries and `exists()` for complex filtering
- **Joins**: Leverage relationships and explicit joins for related data

### Transaction Management
- **Error Handling**: Always use try/except blocks for database operations
- **Rollbacks**: Use `session.rollback()` to undo failed transactions
- **Context Managers**: Use `session.begin()` for automatic transaction management
- **Session Management**: Always close sessions to free resources

### Performance Considerations
- **Bulk Operations**: Use `bulk_save_objects()` and `update()` for large datasets
- **Query Optimization**: Use appropriate loading strategies and joins
- **Indexing**: Consider database indexes for frequently queried columns

## Next Steps

You're now ready to move on to:
- **Topic 4**: Database Migrations with Alembic
- **Module 5**: SQLAlchemy Advanced Features
- **Module 6**: Building Data APIs with FastAPI

Great job mastering CRUD operations! 🎉
